In [1]:
import pulp
import pandas as pd
import numpy as np

class PuLP_Solver_Capsule:
    def __init__(self, name="MCM_Optimization", sense='min'):
        """
        初始化优化器
        sense: 'max' 或 'min'
        """
        self.sense = pulp.LpMaximize if sense == 'max' else pulp.LpMinimize
        self.prob = pulp.LpProblem(name, self.sense)
        self.vars = {} 
        self.is_solved = False
        self.original_cats = {} # [NEW] 用于存储变量原始类型，确保回滚安全

    def add_variables(self, names, low_bound=0, up_bound=None, cat='Integer'):
        """添加变量并记录类型"""
        cat_map = {'Continuous': pulp.LpContinuous, 'Integer': pulp.LpInteger, 'Binary': pulp.LpBinary}
        pulp_cat = cat_map.get(cat, pulp.LpContinuous)
        
        # 创建变量
        new_vars = pulp.LpVariable.dicts("Var", names, lowBound=low_bound, upBound=up_bound, cat=pulp_cat)
        self.vars.update(new_vars)
        
        # [NEW] 记录原始类型，以便后续松弛操作后恢复
        for k, v in new_vars.items():
            self.original_cats[v.name] = pulp_cat
        return new_vars

    def add_constraint(self, expr, sense, rhs, name):
        """添加约束"""
        if sense == '<=': c = (expr <= rhs)
        elif sense == '>=': c = (expr >= rhs)
        elif sense == '==': c = (expr == rhs)
        else: raise ValueError("Sense error")
        self.prob += c, name

    def set_objective(self, expr):
        """直接设置目标函数 (建议在外部构建好表达式传入)"""
        self.prob += expr

    def solve(self, time_limit=60):
        """[NEW] 求解 (增加防卡死限制)"""
        print(f"\n🚀 开始求解 (限时 {time_limit}s)...")
        # msg=0 关闭底层日志，保持界面清爽; timeLimit 防止大规模问题卡死
        solver = pulp.PULP_CBC_CMD(msg=0, timeLimit=time_limit)
        status = self.prob.solve(solver)
        self.is_solved = True
        
        # 状态检查与 Debug
        status_str = pulp.LpStatus[status]
        if status != 1: # Not Optimal
            print(f"⚠️ 警告: 求解状态为 {status_str}")
            if status == -1: # Infeasible (无解)
                print(">>> 正在导出冲突模型 'Debug_Infeasible.lp'，请用文本编辑器打开检查约束方向！")
                self.prob.writeLP("Debug_Infeasible.lp")
        else:
            print(f"✅ 求解成功! Obj = {pulp.value(self.prob.objective):,.2f}")
            
        return status_str

    def solve_relaxed(self):
        """
        【V2.0 核心黑科技】原子化线性松弛分析
        无需重写代码，一键完成：MIP -> LP -> Shadow Price -> MIP
        """
        print("\n🧬 启动线性松弛分析 (Shadow Price Analysis)...")
        
        # 1. 临时修改变量类型 (Integer -> Continuous)
        for v in self.prob.variables():
            v.cat = pulp.LpContinuous

        # 2. 求解 (线性规划通常很快，不需要太长 timeout)
        self.prob.solve(pulp.PULP_CBC_CMD(msg=0))
        
        # 3. 提取分析结果
        shadow_data = []
        if self.prob.status == 1:
            for name, c in self.prob.constraints.items():
                # 只记录有价值的非零影子价格
                if abs(c.pi) > 1e-6: 
                    shadow_data.append({'Constraint': name, 'Shadow_Price': c.pi, 'Slack': c.slack})
            print(f"✅ 松弛求解成功，提取到 {len(shadow_data)} 条关键边际信息。")
        else:
            print("⚠️ 松弛模型无解，无法分析。")

        # 4. [关键] 恢复变量类型 (回滚现场)
        for v in self.prob.variables():
            if v.name in self.original_cats:
                 v.cat = self.original_cats[v.name]
        
        # 按绝对值排序，直接展示最大瓶颈
        return pd.DataFrame(shadow_data).sort_values(by='Shadow_Price', key=abs, ascending=False)

    def export_audit_matrix(self):
        """导出审计矩阵"""
        data = []
        for name, c in self.prob.constraints.items():
            row = {'_Constraint': name, '_RHS': -c.constant}
            for var, coeff in c.items():
                row[var.name] = coeff
            data.append(row)
        return pd.DataFrame(data).fillna(0)
    
    def get_results_df(self):
        """[FIX] 获取结果，不依赖字符串替换，更稳健"""
        if not self.is_solved: return None
        # 直接利用 self.vars 的 key (原始 Route_ID)，避免 replace('Var_', '') 可能带来的误伤
        data = []
        for key, var in self.vars.items():
            data.append({'Variable': key, 'Value': var.varValue})
        return pd.DataFrame(data)